In [1]:
# ===========================================> Carga librerías <====================================================

import pandas as pd
import numpy as np
import pickle

# Transformación de datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
import category_encoders as ce

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Seleccion de variables y tuning de hiperparámetros
from sklearn.feature_selection import RFECV, RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Métricas para evaluar un modelo de clasificación
from sklearn.metrics import classification_report, precision_recall_curve, auc, roc_curve, roc_auc_score, average_precision_score, confusion_matrix

# Librerías para visualización de resultados
import matplotlib.pyplot as plt
import seaborn as sns

# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesado y modelado
# ------------------------------------------------------------------------------
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz, export_text
from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import accuracy_score, confusion_matrix, auc, plot_roc_curve, roc_curve, classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, auc, RocCurveDisplay , roc_curve, classification_report

from scipy.stats import randint, uniform


# Para que no se corten el listado de filas y columnas al ejecutar instrucciones
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# ===========================================> Carga Datos <========================================================

datos = pd.read_csv('../../data/raw/train.csv', low_memory=False)

In [3]:
# ===========================================> Definir variables <==================================================

IDENTIFIER = "MachineIdentifier"
LABEL = "HasDetections"

cat_cols = []
num_cols = []

datos[IDENTIFIER] = datos[IDENTIFIER].astype("category")
datos[LABEL] = datos[LABEL].astype("float64")

param_test = {
    'n_estimators' : np.arange(20,100,5),
    'feature_fraction' : np.arange(0.1,1,0.1),
    'bagging_fraction' : np.arange(0.5,0.99,0.05),
    'num_leaves' : np.arange(30,100,10),
    'learning_rate' : np.arange(0.01,0.5,0.05),
    'max_depth' : np.arange(3,50,2),
    'min_child_samples' : np.arange(2,7,1),
    'reg_alpha' : np.arange(0,1,0.2),
    'reg_lambda' : np.arange(0,1,0.2),
    'colsample_bytree' : np.arange(0,1,0.2)
    }   

In [4]:
# ===========================================> Datos <==============================================================

X_train, X_test, y_train, y_test = train_test_split(
                                        datos.drop(columns = LABEL)[cat_cols+num_cols],
                                        datos[LABEL],
                                        random_state = 123)

In [5]:
modelo = lgb.LGBMClassifier(objective="binary", n_jobs=-1)

In [6]:
randomized_search_cv = RandomizedSearchCV(estimator=modelo, param_distributions=param_test, n_iter=20, scoring=('roc_auc'), cv=10,verbose=True)
#randomized_search_cv = RandomizedSearchCV(estimator=modelo, param_distributions=param_test, n_iter=100, scoring=('roc_auc'), cv=5,verbose=True)

In [7]:
fit_params = {
    'early_stopping_rounds': 80,
    'eval_metric': 'roc_auc',
    'eval_set': [(X_test, y_test)],
    'eval_names' : ["valid"]
}

In [8]:
randomized_search_cv = randomized_search_cv.fit(X_train, y_train, **fit_params)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.2 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.6000000000000001, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6000000000000001
[1]	valid's binary_logloss: 0.693147
[2]	valid's binary_logloss: 0.693147
[3]	valid's binary_logloss: 0.693147
[4]	valid's binary_logloss: 0.693147
[5]	valid's binary_logloss: 0.693147
[6]	valid's binary_logloss: 0.693147
[7]	valid's binary_logloss: 0.693147
[8]	valid's binary_logloss: 0.693147
[9]	valid's binary_logloss: 0.693147
[10]	valid's binary_logloss: 0.693147
[11]	valid's binary_logloss: 0.693147
[12]	valid's binary_logloss: 0.693147
[13]	valid's binary_logloss: 0.693147
[14]	valid's binary_logloss: 0.693147
[15]	valid's binary_logloss: 0.693147
[16]	valid's binary_logloss: 0.693147
[17]	valid's binary_logloss: 0.693147
[18]	valid's binary_logloss: 

c:\Python311\Lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[24]	valid's binary_logloss: 0.693147
[25]	valid's binary_logloss: 0.693147
[26]	valid's binary_logloss: 0.693147
[27]	valid's binary_logloss: 0.693147
[28]	valid's binary_logloss: 0.693147
[29]	valid's binary_logloss: 0.693147
[30]	valid's binary_logloss: 0.693147
[31]	valid's binary_logloss: 0.693147
[32]	valid's binary_logloss: 0.693147
[33]	valid's binary_logloss: 0.693147
[34]	valid's binary_logloss: 0.693147
[35]	valid's binary_logloss: 0.693147
[36]	valid's binary_logloss: 0.693147
[37]	valid's binary_logloss: 0.693147
[38]	valid's binary_logloss: 0.693147
[39]	valid's binary_logloss: 0.693147
[40]	valid's binary_logloss: 0.693147
[41]	valid's binary_logloss: 0.693147
[42]	valid's binary_logloss: 0.693147
[43]	valid's binary_logloss: 0.693147
[44]	valid's binary_logloss: 0.693147
[45]	valid's binary_logloss: 0.693147
[46]	valid's binary_logloss: 0.693147
[47]	valid's binary_logloss: 0.693147
[48]	valid's binary_logloss: 0.693147
[49]	valid's binary_logloss: 0.693147
[50]	valid's

c:\Python311\Lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[25]	valid's binary_logloss: 0.693147
[26]	valid's binary_logloss: 0.693147
[27]	valid's binary_logloss: 0.693147
[28]	valid's binary_logloss: 0.693147
[29]	valid's binary_logloss: 0.693147
[30]	valid's binary_logloss: 0.693147
[31]	valid's binary_logloss: 0.693147
[32]	valid's binary_logloss: 0.693147
[33]	valid's binary_logloss: 0.693147
[34]	valid's binary_logloss: 0.693147
[35]	valid's binary_logloss: 0.693147
[36]	valid's binary_logloss: 0.693147
[37]	valid's binary_logloss: 0.693147
[38]	valid's binary_logloss: 0.693147
[39]	valid's binary_logloss: 0.693147
[40]	valid's binary_logloss: 0.693147
[41]	valid's binary_logloss: 0.693147
[42]	valid's binary_logloss: 0.693147
[43]	valid's binary_logloss: 0.693147
[44]	valid's binary_logloss: 0.693147
[45]	valid's binary_logloss: 0.693147
[46]	valid's binary_logloss: 0.693147
[47]	valid's binary_logloss: 0.693147
[48]	valid's binary_logloss: 0.693147
[49]	valid's binary_logloss: 0.693147
[50]	valid's binary_logloss: 0.693147
[LightGBM] [

...

c:\Python311\Lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.2 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.6000000000000001, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6000000000000001
[1]	valid's binary_logloss: 0.693147
[2]	valid's binary_logloss: 0.693147
[3]	valid's binary_logloss: 0.693147
[4]	valid's binary_logloss: 0.693147
[5]	valid's binary_logloss: 0.693147
[6]	valid's binary_logloss: 0.693147
[7]	valid's binary_logloss: 0.693147
[8]	valid's binary_logloss: 0.693147
[9]	valid's binary_logloss: 0.693147
[10]	valid's binary_logloss: 0.693147
[11]	valid's binary_logloss: 0.693147
[12]	valid's binary_logloss: 0.693147
[13]	valid's binary_logloss: 0.693147
[14]	valid's binary_logloss: 0.693147
[15]	valid's binary_logloss: 0.693147
[16]	valid's binary_logloss: 0.693147
[17]	valid's binary_logloss: 0.693147
[18]	valid's binary_logloss: 0.693147
[19]	valid's binary_logloss: 0.693147
[20]	valid's bin

c:\Python311\Lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.2 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.6000000000000001, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6000000000000001
[1]	valid's binary_logloss: 0.693147
[2]	valid's binary_logloss: 0.693147
[3]	valid's binary_logloss: 0.693147
[4]	valid's binary_logloss: 0.693147
[5]	valid's binary_logloss: 0.693147
[6]	valid's binary_logloss: 0.693147
[7]	valid's binary_logloss: 0.693147
[8]	valid's binary_logloss: 0.693147
[9]	valid's binary_logloss: 0.693147
[10]	valid's binary_logloss: 0.693147
[11]	valid's binary_logloss: 0.693147
[12]	valid's binary_logloss: 0.693147
[13]	valid's binary_logloss: 0.693147
[14]	valid's binary_logloss: 0.693147
[15]	valid's binary_logloss: 0.693147
[16]	valid's binary_logloss: 0.693147
[17]	valid's binary_logloss: 0.693147
[18]	valid's binary_logloss: 0.693147
[19]	valid's binary_logloss: 0.693147
[20]	valid's bin

c:\Python311\Lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


In [9]:
modelo_best = randomized_search_cv.best_estimator_
best_params = randomized_search_cv.best_params_

best_params

# Current Best 0.87787
#{'reg_lambda': 0.6000000000000001,
# 'reg_alpha': 0.0,
# 'num_leaves': 40,
# 'n_estimators': 55,
# 'min_child_samples': 2,
# 'max_depth': 50,
# 'learning_rate': 0.26,
# 'feature_fraction': 0.7000000000000001,
# 'colsample_bytree': 0.6000000000000001,
# 'bagging_fraction': 0.5}

{'reg_lambda': 0.2,
 'reg_alpha': 0.8,
 'num_leaves': 30,
 'n_estimators': 50,
 'min_child_samples': 6,
 'max_depth': 47,
 'learning_rate': 0.11,
 'feature_fraction': 0.5,
 'colsample_bytree': 0.2,
 'bagging_fraction': 0.6000000000000001}